In [1]:
# поле 7х7, игроки поочериди выбирают колонку, в которой добавляется клетка их цвета. 
# задача игрока - образовать линию из 4 клеток своего увета по вертикали, горизонтали или диагонали

In [1]:
#!pip install numpy
#!pip install pandas
#!pip install gym
#!pip install matplotlib
#!pip install ray
#!pip install ray[tune]
#!pip install tensorflow
#!pip install tqdm
#!pip install dm_tree
#!pip install scikit-learn
#!pip install scikit-image
#!pip install lz4
#!pip install torch

In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import gym
from copy import deepcopy
#from tqdm.notebook import tqdm
from tqdm import tqdm
from matplotlib import pyplot as plt


In [3]:
import ray
import ray.rllib.algorithms.ppo as ppo
#import ray.rllib.algorithms.dqn as dqn
from ray import tune

In [4]:
n_cols = 7

max_turns = n_cols**2//2+1 # 24-25 ходов (49 клеток, 2 игрока)

In [5]:
from ray.rllib.models.tf.tf_modelv2 import TFModelV2 as base_model
from ray.rllib.models import ModelCatalog
from ray.rllib.models.tf.fcnet import FullyConnectedNetwork as fcn

In [6]:
import tensorflow as tf
import torch
import keras


In [7]:
def register_env(env, env_name, env_config={}):
    tune.register_env(env_name, lambda env_name: env(env_config=env_config))

In [8]:
def default_state():
    state=dict()
    state['mask'] = np.ones(n_cols)
    state['obs'] = np.zeros(n_cols**2, dtype=int)
    return state


In [9]:
def checkmate (desk, col, pos):
    # -1 - игра продолжается
    #  0 - ничья
    #  1 - победа
    res = -1
    comb = [1]*4
    
    # вертикаль - можем смотреть ток вниз
    if pos >=3:
        if np.all(desk[col, pos-3:pos+1] == comb):
            return 1
    
    # горизонталь - до пердела поля
    #r = n_cols - pos - 1
    #l = pos - 1
    for i in range(4):
        if col - i < 0 or col+3+1-i > n_cols: pass
        elif np.all(desk[col-i:col+3+1-i, pos] == comb):
            return 1
        
    # диагональ - все 4 стороны
    for i in range(4):
        if col - i < 0 or col+3+1-i > n_cols: pass
        elif pos - i < 0 or pos+3+1-i > n_cols: pass
        elif np.all(desk[col-i:col+3+1-i, pos-i:pos+3+1-i] == comb):
            return 1
    #print((desk==0).sum())
    if (desk==0).sum() == 0:
        res = 0
    return res

In [10]:
def checkmate (desk, col, pos):
    # -1 - игра продолжается
    #  0 - ничья
    #  1 - победа
    res = -1
    comb = [1]*4
    
    # вертикаль - можем смотреть ток вниз
    if pos >=3:
        if np.all(desk[col, pos-3:pos+1] == comb):
            return 1
    
    # горизонталь - до пердела поля
    #r = n_cols - pos - 1
    #l = pos - 1
    for i in range(4):
        if col - i < 0 or col+3+1-i > n_cols: pass
        elif np.all(desk[col-i:col+3+1-i, pos] == comb):
            return 1
        
    # диагональ - все 4 стороны
    for i in range(4):
        if   col - i   < 0 or col+3+1-i > n_cols: pass
        elif pos - i   < 0 or pos+3+1-i > n_cols: pass
        elif col-(3-i) < 0 or col+i+1   > n_cols: pass
        elif pos-(3-i) < 0 or pos+i+1   > n_cols: pass
        elif np.all(desk[range(col-i,col+(3-i)+1), range(pos-i,pos+(3-i)+1)] == comb) \
          or np.all(desk[range(col-(3-i),col+i+1), range(pos-(3-i),pos+i+1)] == comb):
            return 1
    #print((desk==0).sum())
    if (desk==0).sum() == 0:
        res = 0
    return res

In [11]:
def place(desk, col):
    pos = np.where(desk[col]==0)[0]
    #print('pos:', pos)
    desk[col][pos[0]] = 1
  
    check = checkmate(desk, col, pos[0])
    #print(desk, '\n')
    
    return desk, check

In [12]:
def switch(desk):
    desk = deepcopy(desk)
    desk[desk ==  1] = -1
    desk[desk ==  2] =  1
    desk[desk == -1] =  2
    return desk

In [13]:
from gym.spaces import Box, Tuple, Dict, Discrete, MultiDiscrete, MultiBinary

In [14]:
class MyEnv(gym.Env):

    def __init__(self, env_config={}):
        self.action_space = Discrete(n_cols)
        self.observation_space = Dict({'mask': MultiBinary(n_cols),
                                       'obs':  MultiDiscrete([3]*n_cols**2)})
        self.reward = 0
        self.done =   False
        self.state =  default_state()
        
        self.opp = tf.keras.Sequential([
                   tf.keras.layers.Dense(256, activation='sigmoid',input_shape = (147,)),
                   tf.keras.layers.Dense(256, activation='sigmoid'),
                   #tf.keras.layers.Dense(n_cols),
                   tf.keras.layers.Dense(n_cols, activation='softmax'),    
                   ])
        self.pp = ray.rllib.models.preprocessors.OneHotPreprocessor(MultiDiscrete([3]*n_cols**2))

    def update_weights(self, weights):
        self.opp.set_weights(weights)
        
    def make_opposite_action(self, obs, mask):
        #print(self.pp.transform(switch(obs).flatten()))

        act_probs = self.opp(self.pp.transform(switch(obs).flatten()).reshape(1, -1))
        #print('act_probs:', act_probs)
        
        avail_acts = np.arange(n_cols)[mask==1]
        #print('avail_acts:', avail_acts)
        
        avail_probs = act_probs.numpy().flatten()[mask==1]
        #print('avail_probs:', avail_probs)
        
        act_probs_avail = np.exp(avail_probs)/np.sum(np.exp(avail_probs))
        #print('act_probs_avail', list(act_probs_avail))
        
        act = np.random.choice(avail_acts, p=act_probs_avail)
        #act = avail_acts[np.argmax(avail_probs)]
        #print('act',act)
        
        
        #avail_acts = np.arange(n_cols)[self.state['mask']==1]
        #act = np.random.choice(avail_acts)
        return act

    def reset(self):

        self.reward = 0
        self.done = False
        self.state = default_state()

        return self.state


    def step(self, action):
        
        self.reward = 0
        #print('act1:', action)
        obs = self.state['obs'].reshape(n_cols, n_cols)

        obs, check = place(obs, action)
        if check != -1: 
            self.done=True
            if check == 1:self.reward = 1
            else: self.reward = 0

        #print('mask before act1:', self.state['mask'])
        if (obs[action] != 0).all():
            self.state['mask'][action] = 0
        #print('mask after act1:', self.state['mask'])
        #print(obs)
        #print()
            

        if not self.done:

            act2 = self.make_opposite_action(obs, self.state['mask'])
            #print(act2)
           
            obs, check = place(switch(obs), act2)
            obs = switch(obs)
            #print(obs)
            
            if check != -1: 
                self.done=True
                if check == 1:self.reward = -1
                else: self.reward = 0
                    
            #print('mask before act2:', self.state['mask'])
            if (obs[act2] != 0).all():
                self.state['mask'][act2] = 0
            #print('mask after act2:', self.state['mask'])

        self.state['obs'] =  obs.flatten()
        #print()
        return self.state, self.reward, self.done, {}

In [15]:
class Agent(base_model):
    def __init__(self, obs_space, action_space, num_outputs,
        model_config, name, true_obs_shape=(n_cols**2*3,),
        action_embed_size=n_cols, *args, **kwargs):
        
        super(Agent, self).__init__(obs_space,
            action_space, num_outputs, model_config, name, 
            *args, **kwargs)
        
        self.action_embed_model = fcn(
            gym.spaces.Box(-1, 1, shape=true_obs_shape, dtype=int), 
                action_space, action_embed_size,
            model_config, name + "_action_embedding")
        
    def forward(self, input_dict, state, seq_lens):
        #print(type(input_dict))
        action_mask = input_dict["obs"]["mask"]

        # Compute the predicted action embedding
        action_embed, _ = self.action_embed_model({
            "obs": input_dict["obs"]["obs"]})

        # Mask out invalid actions 
        inf_mask = tf.maximum(tf.math.log(action_mask), tf.float32.min)
        return action_embed + inf_mask, state

    def value_function(self):
        return self.action_embed_model.value_function()


In [16]:
def train(agent, n_iter, n_report):
    for i in tqdm(range(n_iter)):
        report = agent.train()
        if not (i % n_report):  
            
            print()
            print('mean episode length:', report['episode_len_mean'])
            print('max episode reward:',  report['episode_reward_max'])
            print('mean episode reward:', report['episode_reward_mean'])
            print('min episode reward:',  report['episode_reward_min'])
            print('total episodes:',      report['episodes_total'])
            print()

            checkpoint = agent.save()
            #print("checkpoint saved at", checkpoint)

In [18]:
ray.shutdown()
ray.init()

trainer_config = {
        "model": {
            "custom_model": "custom_agent",
            #'use_lstm':True,
            },
        "framework":'tf2',
        #'eager_tracing':True,
        "num_workers": 6,
        'lr': .03,
        'gamma': 1,
        
        #'train_batch_size':100
         }

ModelCatalog.register_custom_model('custom_agent', Agent)
register_env(MyEnv, 'custom_env', env_config={})
trainer = ppo.PPO(env='custom_env', config=trainer_config)
#trainer = dqn.DQN(env='custom_env', config=trainer_config)


n_epochs = 100000
ipe = 1 # iterations per epoch
rpt = 1

for i in range(n_epochs):
    train(trainer, ipe+i//2, rpt)
    #вытаскиваем веса обученной модели и присваием в сеть-соперника в каждого агента
    weights = np.array(trainer.get_weights()['default_policy'], dtype='object')[[0,1,4,5,8,9]]
    trainer.workers.foreach_env(lambda x: x.update_weights(weights))
    

2022-11-28 15:54:05,163	INFO worker.py:1518 -- Started a local Ray instance.
2022-11-28 15:54:06,242	WARNING ppo.py:350 -- `train_batch_size` (4000) cannot be achieved with your other settings (num_workers=6 num_envs_per_worker=1 rollout_fragment_length=200)! Auto-adjusting `rollout_fragment_length` to 667.
(RolloutWorker pid=13680) 2022-11-28 15:54:09,564	WARNING env.py:142 -- Your env doesn't have a .spec.max_episode_steps attribute. This is fine if you have set 'horizon' in your config dictionary, or `soft_horizon`. However, if you haven't, 'horizon' will default to infinity, and your environment will not be reset.
  0%|                                                     | 0/1 [00:00<?, ?it/s](RolloutWorker pid=13680) /home/zfear/Documents/software/jupyter/lib/python3.8/site-packages/numpy/core/_methods.py:180: RuntimeWarning: overflow encountered in reduce
(RolloutWorker pid=13680)   ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
100%|█████████████████████████████████


mean episode length: 11.927927927927929
max episode reward: 1.0
mean episode reward: 0.23723723723723725
min episode reward: -1.0
total episodes: 333



100%|█████████████████████████████████████████████| 1/1 [00:29<00:00, 29.81s/it]



mean episode length: 9.155606407322654
max episode reward: 1.0
mean episode reward: 0.28604118993135014
min episode reward: -1.0
total episodes: 770



 50%|██████████████████████▌                      | 1/2 [00:30<00:30, 30.61s/it]

...


mean episode length: 7.182468694096601
max episode reward: 1.0
mean episode reward: 0.6457960644007156
min episode reward: -1.0
total episodes: 84748



 67%|█████████████████████████████▎              | 8/12 [03:59<02:00, 30.12s/it]


mean episode length: 7.305147058823529
max episode reward: 1.0
mean episode reward: 0.6507352941176471
min episode reward: -1.0
total episodes: 85292



 67%|█████████████████████████████▎              | 8/12 [04:03<02:01, 30.45s/it]


KeyboardInterrupt: 

In [19]:
def visualize(obs):
    matrix = env.state['obs'].reshape(n_cols, n_cols).T[::-1].astype(str)
    matrix[np.where(matrix=='0')] = '░'
    matrix[np.where(matrix=='1')] = '▒'
    matrix[np.where(matrix=='2')] = '▓'
    print(matrix, '\n')

In [23]:
# тест  обученности
env = MyEnv()
env.act=1
visualize(env.state['obs'])
while not env.done:
    
    #env.done=True
    if env.act==1:
        action = int(input())
    else: 
        action = env.make_opposite_action(env.state)
    state, reward, done, _ = env.step(action)
    visualize(env.state['obs'])


[['░' '░' '░' '░' '░' '░' '░']
 ['░' '░' '░' '░' '░' '░' '░']
 ['░' '░' '░' '░' '░' '░' '░']
 ['░' '░' '░' '░' '░' '░' '░']
 ['░' '░' '░' '░' '░' '░' '░']
 ['░' '░' '░' '░' '░' '░' '░']
 ['░' '░' '░' '░' '░' '░' '░']] 

5
[['░' '░' '░' '░' '░' '░' '░']
 ['░' '░' '░' '░' '░' '░' '░']
 ['░' '░' '░' '░' '░' '░' '░']
 ['░' '░' '░' '░' '░' '░' '░']
 ['░' '░' '░' '░' '░' '░' '░']
 ['░' '░' '░' '░' '░' '▓' '░']
 ['░' '░' '░' '░' '░' '▒' '░']] 

5
[['░' '░' '░' '░' '░' '░' '░']
 ['░' '░' '░' '░' '░' '░' '░']
 ['░' '░' '░' '░' '░' '░' '░']
 ['░' '░' '░' '░' '░' '░' '░']
 ['░' '░' '░' '░' '░' '▒' '░']
 ['░' '░' '░' '░' '░' '▓' '░']
 ['░' '░' '▓' '░' '░' '▒' '░']] 

5
[['░' '░' '░' '░' '░' '░' '░']
 ['░' '░' '░' '░' '░' '░' '░']
 ['░' '░' '░' '░' '░' '░' '░']
 ['░' '░' '░' '░' '░' '▒' '░']
 ['░' '░' '░' '░' '░' '▒' '░']
 ['░' '░' '▓' '░' '░' '▓' '░']
 ['░' '░' '▓' '░' '░' '▒' '░']] 

5
[['░' '░' '░' '░' '░' '░' '░']
 ['░' '░' '░' '░' '░' '░' '░']
 ['░' '░' '░' '░' '░' '▒' '░']
 ['░' '░' '░' '░' '

###### 